![](https://images.aicrowd.com/raw_images/challenges/banner_file/1023/a8ebe297c369ea4a7697.png)

<h2><center>Starter Code for Sentiment Classification</center></h2>


In this baseline we will be training an sklearn model to do a multi-class classificattion of sentiment from face embeddings. 

## Downloading Dataset

Installing puzzle datasets via `aicrowd-cli`

In [1]:
# !pip install aicrowd-cli

# Make sure to re-run below code whenever you restart colab notebook
#%load_ext aicrowd.magic

In [2]:
# # Logging in from our AIcrowd account. Make sure you have accepted the puzzle rules before logging in!  

#%aicrowd login

In [3]:
# # Creating a new data directory and downloading the dataset 

# !rm -rf data
# !mkdir data
# %aicrowd ds dl -c sentiment-classification -o data

## Importing Libraries

In this baseline, we will be sing sklearn [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to classify the sentiment of face embeddings. 

In [4]:
import pandas as pd
import os
import numpy as np
from ast import literal_eval
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score

random.seed(42)

## Reading Dataset

As mented in the challenge readme, we have three different sets provided - train, validation and test respectively.  

In [5]:
# Readging the csv 

train = pd.read_csv("data/train.csv")
val = pd.read_csv("data/val.csv")
submission = pd.read_csv("data/sample_submission.csv")

train

,embeddings,label
0,"[0.3206779360771179, 0.988215982913971, 1.0441...",positive
1,"[0.05074610561132431, 1.0742985010147095, 0.60...",negative
2,"[0.41962647438049316, 0.4505457878112793, 1.39...",negative
3,"[0.4361684024333954, 0.19191382825374603, 0.83...",positive
4,"[0.6382085084915161, 0.8352395296096802, 0.393...",neutral
...,...,...
4995,"[2.2057647705078125, 1.1072001457214355, 0.435...",neutral
4996,"[0.6344252228736877, 1.164398193359375, 0.7155...",negative
4997,"[0.9160683155059814, 0.39996421337127686, 0.82...",negative
4998,"[0.006456990726292133, 0.18667978048324585, 0....",positive


In [6]:
# Getting the feature and labels from each set. 


X = np.array([literal_eval(embedding)  for embedding in train['embeddings'].values])
y = np.array(train['label'].values)

X_val = np.array([literal_eval(embedding)  for embedding in val['embeddings'].values])
y_val = np.array(val['label'].values)

In [7]:
X.shape, y.shape, X_val.shape, y.shape

((5000, 512), (5000,), (2000, 512), (5000,))

## Training the model

Here, we will be training our model using the training set. 

In [8]:
# model = RandomForestClassifier()
# model

In [9]:
# model.fit(X, y)

### Testing the Model

Here, we will be evaluator our model using validation set 

In [10]:
# y_pred = model.predict(X_val)

# print(f"F1 Score : {f1_score(y_val, y_pred, average='weighted')}")
# print(f"Accuracy Score : {accuracy_score(y_val, y_pred)}")

# TF Keras

In [11]:
import tensorflow as tf

In [12]:
from datetime import datetime

In [13]:
import optuna
from optuna.trial import TrialState
from optuna.integration import TFKerasPruningCallback, TensorBoardCallback


In [14]:
y[y=='positive']=2
y[y=='negative']=0
y[y=='neutral']=1
y = tf.one_hot(y,3)
y[:10]

<tf.Tensor: shape=(10, 3), dtype=float32, numpy=
array([[0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.]], dtype=float32)>

In [15]:
np.unique(y,return_counts=True)

(array([0., 1.], dtype=float32), array([10000,  5000], dtype=int64))

In [16]:
y_val[y_val=='positive']=2
y_val[y_val=='negative']=0
y_val[y_val=='neutral']=1
y_val = tf.one_hot(y_val,3)
y_val[:10]

<tf.Tensor: shape=(10, 3), dtype=float32, numpy=
array([[0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.]], dtype=float32)>

In [17]:
# y=y.astype(np.float32)
# y_val=y_val.astype(np.float32)

AttributeError: 
        'EagerTensor' object has no attribute 'astype'.
        If you are looking for numpy-related methods, please run the following:
        from tensorflow.python.ops.numpy_ops import np_config
        np_config.enable_numpy_behavior()

In [18]:

# def getModelOld(neurons0=1024,neurons1=1024,neurons2=1024, layers1=3, layers2=3,
#              dropout=0.5, lr=3e-3, ln=0, noise=1, num_heads=8, num_atts=1):
#     x = tf.keras.layers.InputLayer(input_shape=(512,))
    
#     ls=[]
#     for _ in range(num_atts):
#         ls.append(tf.keras.layers.MultiHeadAttention(num_heads, key_dim=1, dropout=dropout))
    
    
#     ls.append(tf.keras.layers.Dense(neurons0, activation='relu'))
    
#     if noise>0:
#         ls.append(tf.keras.layers.GaussianNoise(0.1))
    
#     if ln>0:
#         ls.append(tf.keras.layers.LayerNormalization())
        
#     for _ in range(layers1):
#         ls.append(tf.keras.layers.Dense(neurons1, activation='relu'),)
#     for _ in range(layers2):
#         ls.append(tf.keras.layers.Dense(neurons2, activation='relu'),)
#     if ln>1:
#         ls.append(tf.keras.layers.LayerNormalization())
#     ls.append(tf.keras.layers.Dropout(dropout))
#     ls.append(tf.keras.layers.Dense(3, activation='softmax'))
#     model = tf.keras.models.Sequential(ls)
#     model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
#                   loss='categorical_crossentropy',
#                   metrics=['mae','mse','accuracy',my_f1,my_acc],
#                   run_eagerly=True)
#     return model

# # def getModelCls():
# #     model = tf.keras.models.Sequential([tf.keras.layers.Dense(1024, activation='relu', input_shape=(512,)),
# # #                                         tf.keras.layers.Dropout(0.25),
# #                                         tf.keras.layers.LayerNormalization(),
# #                                         tf.keras.layers.Dense(1024, activation='relu'),
# #                                         tf.keras.layers.Dense(1024, activation='relu'),
# #                                         tf.keras.layers.Dropout(0.5),
# #                                         tf.keras.layers.Dense(1024, activation='relu'),
# #                                         tf.keras.layers.LayerNormalization(),
# #                                         tf.keras.layers.Dropout(0.5),
# #                                         tf.keras.layers.Dense(3, activation='softmax')])
# #     model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
# #                   loss='categorical_crossentropy',
# #                   metrics=['mae','mse','accuracy',my_f1,my_acc],
# #                   run_eagerly=True)
# #     return model
# # model = getModelCls()
# # model.summary()

# # model.fit(X/10.0,y,
# #           validation_data=(X_val/10.0,y_val),
# #           batch_size=1024,
# #           epochs=500,
# #           verbose=0,
# #          callbacks=[tf.keras.callbacks.TensorBoard(
# #     log_dir="logs/0/"+str(datetime.now()).replace(' ','_').replace(':','-')+'/',
# #     histogram_freq=1,
# #     write_graph=True,
# #     update_freq="epoch"),#PlotLossesKerasTF(),
# # #                    tf.keras.callbacks.ModelCheckpoint(
# # #     'checkpoint',
# # #     monitor="val_my_f1",
# # #     verbose=1,
# # #     save_best_only=True,
# # #     save_weights_only=False,
# # #     mode="max",
# # #     save_freq="epoch",),
# #                     tf.keras.callbacks.EarlyStopping(
# #                                     monitor="val_my_f1",
# #                                     patience=100,
# #                                     verbose=1,
# #                                     mode="max",
# #                                     restore_best_weights=True,
# #                                 ),
# #                    tf.keras.callbacks.ReduceLROnPlateau(
# #     monitor="val_my_f1",
# #     factor=0.3,
# #     patience=10,
# #     verbose=1,
# #     mode="max",
# #     min_delta=0.0001,
# #     cooldown=0,
# #     min_lr=1e-10,
# # )])

In [38]:
def my_f1(y_val, y_pred, sample_weight=None):
    return f1_score(tf.argmax(y_val,axis=1), tf.argmax(y_pred,axis=1),average='weighted')

def my_acc(y_val, y_pred, sample_weight=None):
    return accuracy_score(tf.argmax(y_val,axis=1), tf.argmax(y_pred,axis=1))

def getModel(neurons0=1024,neurons1=1024,neurons2=1024, layers1=3, layers2=3,
             dropout=0.5, lr=3e-3, ln=0, noise=1, num_heads=8):
    xx = tf.keras.Input(shape=(512,))
    if num_heads>0:
        x=tf.keras.layers.Reshape((1,512))(xx)
        x=tf.keras.layers.MultiHeadAttention(num_heads, key_dim=1, dropout=dropout)(x,x)
        x=tf.keras.layers.Dense(neurons0, activation='relu')(tf.keras.layers.Flatten()(x))
    else:
        x=tf.keras.layers.Dense(neurons0, activation='relu')(xx)

    x=tf.keras.layers.GaussianNoise(noise)(x)
    
    if ln>0:
        x=tf.keras.layers.LayerNormalization()(x)
        
    for _ in range(layers1):
        x=tf.keras.layers.Dense(neurons1, activation='relu')(x)
    for _ in range(layers2):
        x=tf.keras.layers.Dense(neurons2, activation='relu')(x)
    if ln>1:
        x=tf.keras.layers.LayerNormalization()(x)
    x=tf.keras.layers.Dropout(dropout)(x)
    x=tf.keras.layers.Dense(3, activation='softmax')(x)
    model = tf.keras.models.Model(inputs=xx,outputs=x)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                  loss='categorical_crossentropy',
                  metrics=['mae','mse','accuracy',my_f1,my_acc],
                  run_eagerly=True)
    return model


def getModelHuber(neurons0=1024,neurons1=1024,neurons2=1024, layers1=3, layers2=3,
             dropout=0.5, lr=3e-3, ln=0, noise=1, num_heads=8, num_atts=1):
    xx = tf.keras.Input(shape=(512,))
    if num_atts>0:
        x=tf.keras.layers.Reshape((1,512))(xx)
        x=tf.keras.layers.MultiHeadAttention(num_heads, key_dim=1, dropout=dropout)(x,x)
#         x=tf.keras.layers.Attention()([x,x])

        for _ in range(num_atts-1):
            x=tf.keras.layers.MultiHeadAttention(num_heads, key_dim=1, dropout=dropout)(x,x)
#             x=tf.keras.layers.Attention()([x,x])
        x=tf.keras.layers.Dense(neurons0, activation='relu')(tf.keras.layers.Flatten()(x))
    else:
        x=tf.keras.layers.Dense(neurons0, activation='relu')(xx)

    if noise>0:
        x=tf.keras.layers.GaussianNoise(0.1)(x)
    
    if ln>0:
        x=tf.keras.layers.LayerNormalization()(x)
        
    for _ in range(layers1):
        x=tf.keras.layers.Dense(neurons1, activation='relu')(x)
    for _ in range(layers2):
        x=tf.keras.layers.Dense(neurons2, activation='relu')(x)
    if ln>1:
        x=tf.keras.layers.LayerNormalization()(x)
    x=tf.keras.layers.Dropout(dropout)(x)
    x=tf.keras.layers.Dense(1)(x)
    model = tf.keras.models.Model(inputs=xx,outputs=x)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                  loss='huber',
                  metrics=['mae','mse'],#,'accuracy',my_f1,my_acc],
                  run_eagerly=True)
    return model

# m = getModel(num_atts=0,num_heads=2)
# m.summary()

In [39]:
# layer = tf.keras.layers.MultiHeadAttention(num_heads=2, key_dim=2)
# target = tf.keras.Input(shape=[1,16])
# source = tf.keras.Input(shape=[1,16])
# output_tensor, weights = layer(target, source,
#                                 return_attention_scores=True)
# print(output_tensor.shape)

# print(weights.shape)


In [42]:
max_epochs=100
def objective(trial):
    neurons0 = trial.suggest_int("neurons0", 16, 128, log=False)
    
    
    lr = 1e-2 #trial.suggest_float("lr", 1e-7, 1e-1, log=True)
    dropout = trial.suggest_float("dropout", 0, 0.9)
    layers1 = trial.suggest_int("layers1", 0, 3)
    layers2 = trial.suggest_int("layers2", 0, 3)

    if layers1>0:
        neurons1 = trial.suggest_int("neurons1", 16, 1024, log=False)
    else:
        neurons1=0
        
    if layers2>0:
        neurons2 = trial.suggest_int("neurons2", 128, 2048, log=False)
    else:
        neurons2=0
    ln = trial.suggest_int("layer_norms", 0, 2)
    batch_size = 256#trial.suggest_int("batch_size", 8, 1024)
    noise = trial.suggest_float("noise", 1e-5, 1e-2, log=True)
    
    disable_att = trial.suggest_categorical('disable_att',['disabled','enabled'])
    if disable_att=='disabled':
        num_heads=0
    else:
        num_heads = trial.suggest_int("num_heads", 1, 512)

    model = getModel(neurons0=neurons0,
                     neurons1=neurons1,
                     neurons2=neurons2,
                     layers1=layers1,
                     layers2=layers2,
                     dropout=dropout,
                     lr=lr,
                     ln=ln,
                     noise=noise,
                     num_heads=num_heads)
    
    history = model.fit(X/10.0,y,
              validation_data=(X_val/10.0,y_val),
              batch_size=batch_size,
              epochs=max_epochs,
              verbose=0,
             callbacks=[tf.keras.callbacks.TensorBoard(
                            log_dir="logs4/optuna/"+str(datetime.now()).replace(' ','_').replace(':','-')+'/',
                            histogram_freq=1,
                            write_graph=True,
                            update_freq="epoch"),
                        TFKerasPruningCallback(trial, "val_loss"),
                        tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.3,
        patience=20,
        verbose=0,
        mode="max",
        min_delta=0.0001,
        cooldown=0,
        min_lr=1e-10,

    )])
    
    return max(history.history['val_loss'])

In [43]:
study = optuna.create_study(direction="minimize",
                            pruner=optuna.pruners.PatientPruner(optuna.pruners.HyperbandPruner(
        min_resource=1, max_resource=max_epochs, reduction_factor=3), patience=25))
study.optimize(objective, callbacks=[TensorBoardCallback('logs4/optuna/','val_loss')])

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: ExperimentalWarning: PatientPruner is experimental (supported from v2.8.0). The interface can change in the future.
  This is separate from the ipykernel package so we can avoid doing imports until
[I 2022-02-14 14:25:58,026] A new study created in memory with name: no-name-c3446e30-e20d-479e-b41f-f4f8ba203445
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: ExperimentalWarning: TensorBoardCallback is experimental (supported from v2.0.0). The interface can change in the future.
  after removing the cwd from sys.path.
[I 2022-02-14 14:31:36,870] Trial 0 finished with value: 2.3406832218170166 and parameters: {'neurons0': 63, 'dropout': 0.7693871997161831, 'layers1': 3, 'layers2': 1, 'neurons1': 262, 'neurons2': 1371, 'layer_norms': 2, 'noise': 0.0006250441660476801, 'disable_att': 'disabled'}. Best is trial 0 with value: 2.3406832218170166.
[I 2022-02-14 14:35:51,290] Trial 1 finished with value: 0.77

[I 2022-02-14 18:35:15,086] Trial 50 pruned. Trial was pruned at epoch 66.
[I 2022-02-14 18:41:45,285] Trial 51 finished with value: 0.9023485779762268 and parameters: {'neurons0': 62, 'dropout': 0.7190833503746795, 'layers1': 1, 'layers2': 0, 'neurons1': 166, 'layer_norms': 0, 'noise': 0.0011785708436385493, 'disable_att': 'enabled', 'num_heads': 98}. Best is trial 15 with value: 0.7515884637832642.
[I 2022-02-14 18:47:37,401] Trial 52 pruned. Trial was pruned at epoch 81.
[I 2022-02-14 18:52:00,288] Trial 53 pruned. Trial was pruned at epoch 62.
[I 2022-02-14 18:55:49,040] Trial 54 pruned. Trial was pruned at epoch 63.
[I 2022-02-14 18:58:25,243] Trial 55 pruned. Trial was pruned at epoch 47.
[I 2022-02-14 19:02:15,073] Trial 56 pruned. Trial was pruned at epoch 47.
[I 2022-02-14 19:04:35,223] Trial 57 pruned. Trial was pruned at epoch 37.
[I 2022-02-14 19:06:40,355] Trial 58 pruned. Trial was pruned at epoch 31.
[I 2022-02-14 19:11:54,432] Trial 59 pruned. Trial was pruned at epoch 

[I 2022-02-14 23:03:10,017] Trial 101 pruned. Trial was pruned at epoch 81.
[I 2022-02-14 23:07:25,389] Trial 102 pruned. Trial was pruned at epoch 51.
[I 2022-02-14 23:14:49,023] Trial 103 finished with value: 0.9641996026039124 and parameters: {'neurons0': 67, 'dropout': 0.7817851385670632, 'layers1': 1, 'layers2': 0, 'neurons1': 88, 'layer_norms': 0, 'noise': 0.0003232355739042078, 'disable_att': 'enabled', 'num_heads': 157}. Best is trial 15 with value: 0.7515884637832642.
[I 2022-02-14 23:19:28,633] Trial 104 pruned. Trial was pruned at epoch 65.
[I 2022-02-14 23:22:57,698] Trial 105 pruned. Trial was pruned at epoch 42.
[I 2022-02-14 23:29:33,167] Trial 106 pruned. Trial was pruned at epoch 66.
[I 2022-02-14 23:33:53,171] Trial 107 pruned. Trial was pruned at epoch 57.
[I 2022-02-14 23:38:12,522] Trial 108 pruned. Trial was pruned at epoch 46.
[I 2022-02-14 23:42:45,135] Trial 109 finished with value: 0.9999175071716309 and parameters: {'neurons0': 40, 'dropout': 0.23589032278156

[I 2022-02-15 04:33:38,394] Trial 159 pruned. Trial was pruned at epoch 64.
[I 2022-02-15 04:42:00,698] Trial 160 pruned. Trial was pruned at epoch 81.
[I 2022-02-15 04:48:32,662] Trial 161 pruned. Trial was pruned at epoch 55.
[I 2022-02-15 04:52:55,614] Trial 162 pruned. Trial was pruned at epoch 48.
[I 2022-02-15 04:56:50,805] Trial 163 pruned. Trial was pruned at epoch 42.
[I 2022-02-15 05:01:04,089] Trial 164 pruned. Trial was pruned at epoch 48.
[I 2022-02-15 05:04:44,545] Trial 165 pruned. Trial was pruned at epoch 34.
[I 2022-02-15 05:10:46,202] Trial 166 pruned. Trial was pruned at epoch 57.
[I 2022-02-15 05:16:26,471] Trial 167 pruned. Trial was pruned at epoch 64.
[I 2022-02-15 05:21:19,441] Trial 168 pruned. Trial was pruned at epoch 48.
[I 2022-02-15 05:26:13,890] Trial 169 pruned. Trial was pruned at epoch 45.
[I 2022-02-15 05:31:14,277] Trial 170 finished with value: 1.0531175136566162 and parameters: {'neurons0': 55, 'dropout': 0.7155058081151544, 'layers1': 1, 'layers2

KeyboardInterrupt: 

# newest submission - HPO enabled attention!

https://www.aicrowd.com/challenges/ai-blitz-xiii/problems/sentiment-classification/submissions/173841

best is trial 15
{'neurons0': 50, 'dropout': 0.46806609816933387, 'layers1': 1, 'layers2': 0, 'neurons1': 23, 'layer_norms': 0, 'noise': 0.0015606054217948873, 'disable_att': 'enabled', 'num_heads': 417}

In [45]:
max_epochs=100
neurons0 = 50
lr = 1e-2 #trial.suggest_float("lr", 1e-7, 1e-1, log=True)
dropout = 0.46806609816933387
layers1 = 1
layers2 = 0

if layers1>0:
    neurons1 = 23
else:
    neurons1=0

if layers2>0:
#     neurons2 = trial.suggest_int("neurons2", 128, 2048, log=False)
    pass
else:
    neurons2=0
ln = 0
batch_size = 256#trial.suggest_int("batch_size", 8, 1024)
noise = 0.0015606054217948873

disable_att = 'enabled'
if disable_att=='disabled':
    num_heads=0
else:
    num_heads = 417

model = getModel(neurons0=neurons0,
                 neurons1=neurons1,
                 neurons2=neurons2,
                 layers1=layers1,
                 layers2=layers2,
                 dropout=dropout,
                 lr=lr,
                 ln=ln,
                 noise=noise,
                 num_heads=num_heads)

history = model.fit(X/10.0,y,
          validation_data=(X_val/10.0,y_val),
          batch_size=batch_size,
          epochs=max_epochs,
          verbose=1,
         callbacks=[tf.keras.callbacks.TensorBoard(
                        log_dir="logs4/optuna-best/"+str(datetime.now()).replace(' ','_').replace(':','-')+'/',
                        histogram_freq=1,
                        write_graph=True,
                        update_freq="epoch"),

                tf.keras.callbacks.EarlyStopping(
                                monitor="val_my_f1",
                                patience=50,
                                verbose=1,
                                mode="max",
                                restore_best_weights=True,
                            ),
                    tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.3,
    patience=20,
    verbose=1,
    mode="max",
    min_delta=0.0001,
    cooldown=0,
    min_lr=1e-10,

)])

history.history['val_loss'], max(history.history['val_loss'])

Epoch 1/100
20/20 [==============================] - 7s 340ms/step - loss: 1.0384 - mae: 0.4231 - mse: 0.2094 - accuracy: 0.4176 - my_f1: 0.3708 - my_acc: 0.4211 - val_loss: 0.9322 - val_mae: 0.3820 - val_mse: 0.1876 - val_accuracy: 0.6220 - val_my_f1: 0.5632 - val_my_acc: 0.6225
Epoch 2/100
20/20 [==============================] - 6s 314ms/step - loss: 0.8860 - mae: 0.3618 - mse: 0.1779 - accuracy: 0.6152 - my_f1: 0.6091 - my_acc: 0.6173 - val_loss: 0.7506 - val_mae: 0.3136 - val_mse: 0.1478 - val_accuracy: 0.6945 - val_my_f1: 0.6737 - val_my_acc: 0.6950
Epoch 3/100
20/20 [==============================] - 6s 298ms/step - loss: 0.7741 - mae: 0.3145 - mse: 0.1559 - accuracy: 0.6512 - my_f1: 0.6336 - my_acc: 0.6490 - val_loss: 0.6551 - val_mae: 0.2874 - val_mse: 0.1294 - val_accuracy: 0.7200 - val_my_f1: 0.7076 - val_my_acc: 0.7201
Epoch 4/100
20/20 [==============================] - 6s 303ms/step - loss: 0.7221 - mae: 0.2916 - mse: 0.1455 - accuracy: 0.6830 - my_f1: 0.6752 - my_acc: 0.

Epoch 30/100
20/20 [==============================] - 6s 297ms/step - loss: 0.4548 - mae: 0.1862 - mse: 0.0902 - accuracy: 0.8146 - my_f1: 0.8118 - my_acc: 0.8150 - val_loss: 0.4970 - val_mae: 0.1878 - val_mse: 0.1014 - val_accuracy: 0.7785 - val_my_f1: 0.7745 - val_my_acc: 0.7779
Epoch 31/100
20/20 [==============================] - 6s 299ms/step - loss: 0.4537 - mae: 0.1834 - mse: 0.0893 - accuracy: 0.8184 - my_f1: 0.8159 - my_acc: 0.8183 - val_loss: 0.4994 - val_mae: 0.1915 - val_mse: 0.1019 - val_accuracy: 0.7845 - val_my_f1: 0.7849 - val_my_acc: 0.7841
Epoch 32/100
20/20 [==============================] - 6s 305ms/step - loss: 0.4461 - mae: 0.1832 - mse: 0.0892 - accuracy: 0.8168 - my_f1: 0.8141 - my_acc: 0.8164 - val_loss: 0.5351 - val_mae: 0.1869 - val_mse: 0.1070 - val_accuracy: 0.7730 - val_my_f1: 0.7739 - val_my_acc: 0.7725
Epoch 33/100
20/20 [==============================] - 6s 312ms/step - loss: 0.4541 - mae: 0.1855 - mse: 0.0900 - accuracy: 0.8212 - my_f1: 0.8198 - my_acc

Epoch 59/100
20/20 [==============================] - 6s 299ms/step - loss: 0.4195 - mae: 0.1698 - mse: 0.0828 - accuracy: 0.8314 - my_f1: 0.8294 - my_acc: 0.8314 - val_loss: 0.5141 - val_mae: 0.1853 - val_mse: 0.1034 - val_accuracy: 0.7805 - val_my_f1: 0.7808 - val_my_acc: 0.7798
Epoch 60/100
20/20 [==============================] - 6s 292ms/step - loss: 0.4072 - mae: 0.1687 - mse: 0.0811 - accuracy: 0.8332 - my_f1: 0.8317 - my_acc: 0.8333 - val_loss: 0.5160 - val_mae: 0.1803 - val_mse: 0.1034 - val_accuracy: 0.7810 - val_my_f1: 0.7794 - val_my_acc: 0.7802
Epoch 61/100
20/20 [==============================] - 6s 310ms/step - loss: 0.4142 - mae: 0.1676 - mse: 0.0819 - accuracy: 0.8342 - my_f1: 0.8310 - my_acc: 0.8329 - val_loss: 0.5074 - val_mae: 0.1837 - val_mse: 0.1026 - val_accuracy: 0.7860 - val_my_f1: 0.7848 - val_my_acc: 0.7854

Epoch 00061: ReduceLROnPlateau reducing learning rate to 2.700000040931627e-05.
Epoch 62/100
20/20 [==============================] - 6s 314ms/step - los

20/20 [==============================] - 6s 301ms/step - loss: 0.4022 - mae: 0.1665 - mse: 0.0800 - accuracy: 0.8348 - my_f1: 0.8330 - my_acc: 0.8349 - val_loss: 0.5171 - val_mae: 0.1803 - val_mse: 0.1035 - val_accuracy: 0.7835 - val_my_f1: 0.7825 - val_my_acc: 0.7827
Epoch 88/100
20/20 [==============================] - 6s 302ms/step - loss: 0.3979 - mae: 0.1627 - mse: 0.0784 - accuracy: 0.8392 - my_f1: 0.8383 - my_acc: 0.8395 - val_loss: 0.5177 - val_mae: 0.1800 - val_mse: 0.1035 - val_accuracy: 0.7835 - val_my_f1: 0.7825 - val_my_acc: 0.7828
Restoring model weights from the end of the best epoch.
Epoch 00088: early stopping


([0.93215012550354,
  0.7506144642829895,
  0.6550961136817932,
  0.6149912476539612,
  0.5660004019737244,
  0.6200670003890991,
  0.5667563080787659,
  0.5406185984611511,
  0.5298784971237183,
  0.5477474331855774,
  0.5173028111457825,
  0.537368893623352,
  0.5290323495864868,
  0.5593916773796082,
  0.5274007320404053,
  0.5282717347145081,
  0.5734229683876038,
  0.5041447281837463,
  0.5077146887779236,
  0.5105924010276794,
  0.5141319036483765,
  0.5009227395057678,
  0.498345285654068,
  0.5021173357963562,
  0.4989565908908844,
  0.5020665526390076,
  0.5090712308883667,
  0.49760156869888306,
  0.5073554515838623,
  0.4969843327999115,
  0.4994041919708252,
  0.5351067185401917,
  0.5006476640701294,
  0.5146024823188782,
  0.5003334879875183,
  0.5038415193557739,
  0.505352258682251,
  0.5022417902946472,
  0.5014651417732239,
  0.5110684633255005,
  0.5076795816421509,
  0.503508448600769,
  0.50431889295578,
  0.5063230395317078,
  0.5075604915618896,
  0.5066820383071

In [46]:
y_pred = model.predict(X_val/10.0)
# y_pred.shape
print(f"F1 Score : {f1_score(tf.argmax(y_val,axis=1), tf.argmax(y_pred,axis=1), average='weighted')}")
print(f"Accuracy Score : {accuracy_score(tf.argmax(y_val,axis=1), tf.argmax(y_pred,axis=1))}")

F1 Score : 0.7866055237424707
Accuracy Score : 0.7875


# Code below is pretty ok

https://www.aicrowd.com/challenges/ai-blitz-xiii/problems/sentiment-classification/submissions/173578

F1 SCORE 0.795

ACUCRACY SCORE 0.794

In [ ]:

neurons0 = 1277
neurons1 = 116
neurons2 = 54
lr = 1.6895e-7
dropout = 0.75541
layers1 = 2
layers2 = 1
ln = 0
batch_size = 175
noise = 0
num_heads = 4
num_atts = 0

model = getModel(neurons0=neurons0,
                 neurons1=neurons1,
                 neurons2=neurons2,
                 layers1=layers1,
                 layers2=layers2,
                 dropout=dropout,
                 lr=lr,
                 ln=ln,
                 noise=noise,
                 num_heads=num_heads,
                 num_atts=num_atts)

history = model.fit(X/10.0,y,
          validation_data=(X_val/10.0,y_val),
          batch_size=batch_size,
          epochs=max_epochs,
          verbose=1,
         callbacks=[tf.keras.callbacks.TensorBoard(
                        log_dir="logs/optuna-att-12/"+str(datetime.now()).replace(' ','_').replace(':','-')+'/',
                        histogram_freq=1,
                        write_graph=True,
                        update_freq="epoch"),

                tf.keras.callbacks.EarlyStopping(
                                monitor="val_my_f1",
                                patience=50,
                                verbose=1,
                                mode="max",
                                restore_best_weights=True,
                            ),                        tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_my_f1",
    factor=0.3,
    patience=20,
    verbose=0,
    mode="max",
    min_delta=0.0001,
    cooldown=0,
    min_lr=1e-10,
)])
    
history.history['val_my_f1']

In [ ]:
y_pred = model.predict(X_val/10.0)
# y_pred.shape
print(f"F1 Score : {f1_score(tf.argmax(y_val,axis=1), tf.argmax(y_pred,axis=1), average='weighted')}")
print(f"Accuracy Score : {accuracy_score(tf.argmax(y_val,axis=1), tf.argmax(y_pred,axis=1))}")

# huber (trial 181) = słabo
https://www.aicrowd.com/challenges/ai-blitz-xiii/problems/sentiment-classification/submissions/173742

In [ ]:
neurons0 = 16
neurons1 = 174
neurons2 = 828
lr = 0.000017281
dropout = 0.15381
layers1 = 2
layers2 = 2
ln = 1
batch_size = 24
noise = 1
num_heads = 6
num_atts = 0

model = getModelHuber(neurons0=neurons0,
                 neurons1=neurons1,
                 neurons2=neurons2,
                 layers1=layers1,
                 layers2=layers2,
                 dropout=dropout,
                 lr=lr,
                 ln=ln,
                 noise=noise,
                 num_heads=num_heads,
                 num_atts=num_atts)

history = model.fit(X/10.0,y,
          validation_data=(X_val/10.0,y_val),
          batch_size=batch_size,
          epochs=max_epochs,
          verbose=1,
         callbacks=[tf.keras.callbacks.TensorBoard(
                        log_dir="logs/optuna-huber2-best/"+str(datetime.now()).replace(' ','_').replace(':','-')+'/',
                        histogram_freq=1,
                        write_graph=True,
                        update_freq="epoch"),

                tf.keras.callbacks.EarlyStopping(
                                monitor="val_mse",
                                patience=50,
                                verbose=1,
                                mode="max",
                                restore_best_weights=True,
                            ),                    tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_mse",
    factor=0.3,
    patience=20,
    verbose=1,
    mode="max",
    min_delta=0.0001,
    cooldown=0,
    min_lr=1e-10,

)])

max(history.history['val_mse'])

In [ ]:
y_pred = (model.predict(X_val/10.0)+0.5).astype(np.uint8)[:,0]
# y_pred.shape
print(f"F1 Score : {f1_score(y_val, y_pred, average='weighted')}")
print(f"Accuracy Score : {accuracy_score(y_val, y_pred)}")

In [ ]:
y_val[:10],y_pred[:10]

In [ ]:
np.count_nonzero(y_pred)

## Generating the Predictions

Generating Predictions from test data to make submission in the puzzle. 

In [47]:
submission_embeddings = [literal_eval(embedding)  for embedding in submission['embeddings'].values]

In [48]:
# with ARGMAX (one hot)
predictions = np.argmax(model.predict(np.array(submission_embeddings)/10.0),axis=1).tolist()

In [ ]:
predictions = (model.predict(np.array(submission_embeddings)/10.0)+0.5).astype(np.uint8)[:,0].tolist()

In [49]:
max(predictions)

2

In [50]:
def convert(a):
    if a==0:
        return 'negative'
    if a==1:
        return 'neutral'
    if a==2:
        return 'positive'

result = list(map(convert, predictions))

In [51]:
np.unique(result, return_counts=True)

(array(['negative', 'neutral', 'positive'], dtype='<U8'),
 array([1000,  964, 1037], dtype=int64))

In [52]:
submission['label'] = result
submission

,embeddings,label
0,"[0.08109518140554428, 0.3090009093284607, 1.36...",positive
1,"[0.6809610724449158, 1.1909409761428833, 0.892...",neutral
2,"[0.14851869642734528, 0.7872061133384705, 0.89...",neutral
3,"[0.44697386026382446, 0.36429283022880554, 0.7...",neutral
4,"[1.8009324073791504, 0.26081395149230957, 0.40...",negative
...,...,...
2996,"[0.9138844609260559, 0.9460961222648621, 0.571...",negative
2997,"[0.7667452096939087, 0.7896291613578796, 0.648...",negative
2998,"[0.8158280849456787, 2.404792070388794, 0.9924...",neutral
2999,"[0.4161085784435272, 0.3146701455116272, 1.139...",positive


### Saving the Predictions

In [53]:
# Saving the predictions
# !rm -rf assets
# !mkdir assets
submission.to_csv(os.path.join("assets", "submission.csv"))

## Submitting our Predictions

In [54]:
%load_ext aicrowd.magic

In [55]:
%aicrowd login

Please login here: https://api.aicrowd.com/auth/GK_bp202k1ZZm_H3yzEIaHUMYINBXXJIb_uoTFE3NI4
API Key valid
Gitlab access token valid
Saved details successfully!


In [56]:
%aicrowd notebook submit -c sentiment-classification -a assets --no-verify

<IPython.core.display.Javascript object>

Using notebook: age_cls.ipynb for submission...
Removing existing files from submission directory...
Scrubbing API keys from the notebook...
An unexpected error occured!
cannot unpack non-iterable NoneType object
To get more information, you can run this command with -v.
To increase level of verbosity, you can go upto -vvvvv


In [ ]:
!ls assets

In [57]:
! cd assets/ ; zip submission.zip submission.csv

The system cannot find the path specified.


Congratulations to making your first submission in the puzzle 🎉  . Let's continue with the journey by improving the baseline & making submission! Don't be shy to ask question related to any errors you are getting or doubts in any part of this notebook in discussion forum or in AIcrowd Discord sever, AIcrew will be happy to help you :)

Have a cool new idea that you want to see in the next blitz ? Let us know!